In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from scipy.stats import mode
# import modelvis
from sklearn.ensemble import RandomForestClassifier
%matplotlib

Using matplotlib backend: MacOSX


## Frame 
Given building details, your task is to build a model that can predict the extent of damage that has been done to a building after an earthquake.

## ACQUIRE
train.csv, test.csv, building_ownership_usage.csv, building_structure.csv, sample_submission.csv

In [7]:
df = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/train.csv')
df_bo = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/Building_Ownership_Use.csv')
df_str = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/building_structure.csv')

In [8]:
df.columns

Index(['area_assesed', 'building_id', 'damage_grade', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id'],
      dtype='object')

## REFINE
Predict damage_grade

In [4]:
df.damage_grade.value_counts()

Grade 5    210825
Grade 4    152244
Grade 3    122288
Grade 2     85084
Grade 1     61320
Name: damage_grade, dtype: int64

In [5]:
df.shape

(631761, 14)

In [6]:
df.dtypes

area_assesed                              object
building_id                               object
damage_grade                              object
district_id                                int64
has_geotechnical_risk                    float64
has_geotechnical_risk_fault_crack          int64
has_geotechnical_risk_flood                int64
has_geotechnical_risk_land_settlement      int64
has_geotechnical_risk_landslide            int64
has_geotechnical_risk_liquefaction         int64
has_geotechnical_risk_other                int64
has_geotechnical_risk_rock_fall            int64
has_repair_started                       float64
vdcmun_id                                  int64
dtype: object

In [7]:
df.isnull().sum()

area_assesed                                 0
building_id                                  0
damage_grade                                 0
district_id                                  0
has_geotechnical_risk                        0
has_geotechnical_risk_fault_crack            0
has_geotechnical_risk_flood                  0
has_geotechnical_risk_land_settlement        0
has_geotechnical_risk_landslide              0
has_geotechnical_risk_liquefaction           0
has_geotechnical_risk_other                  0
has_geotechnical_risk_rock_fall              0
has_repair_started                       33417
vdcmun_id                                    0
dtype: int64

In [8]:
mode(df.has_repair_started)

ModeResult(mode=array([0.]), count=array([409222]))

In [9]:
#Replacing null values by mode(0) value of has_repair_started
df.has_repair_started.fillna(0, inplace=True)

In [10]:
df.isnull().sum()

area_assesed                             0
building_id                              0
damage_grade                             0
district_id                              0
has_geotechnical_risk                    0
has_geotechnical_risk_fault_crack        0
has_geotechnical_risk_flood              0
has_geotechnical_risk_land_settlement    0
has_geotechnical_risk_landslide          0
has_geotechnical_risk_liquefaction       0
has_geotechnical_risk_other              0
has_geotechnical_risk_rock_fall          0
has_repair_started                       0
vdcmun_id                                0
dtype: int64

In [11]:
df.area_assesed.value_counts()

Both                   382992
Building removed       131346
Exterior                99030
Not able to inspect     16575
Interior                 1818
Name: area_assesed, dtype: int64

In [12]:
grade = {'Grade 5': 5, 'Grade 4': 4, 'Grade 3': 3, 'Grade 2': 2, 'Grade 1': 1}
area = {'Both': 0, 'Building removed': 1, 'Exterior': 2, 'Not able to inspect': 3, 'Interior': 4}

In [13]:
df.damage_grade = df.damage_grade.map(grade.get)
df.area_assesed = df.area_assesed.map(area.get)

In [14]:
df.dtypes

area_assesed                               int64
building_id                               object
damage_grade                               int64
district_id                                int64
has_geotechnical_risk                    float64
has_geotechnical_risk_fault_crack          int64
has_geotechnical_risk_flood                int64
has_geotechnical_risk_land_settlement      int64
has_geotechnical_risk_landslide            int64
has_geotechnical_risk_liquefaction         int64
has_geotechnical_risk_other                int64
has_geotechnical_risk_rock_fall            int64
has_repair_started                       float64
vdcmun_id                                  int64
dtype: object

In [16]:
df_bo.columns

Index(['building_id', 'district_id', 'vdcmun_id', 'ward_id',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other'],
      dtype='object')

In [17]:
df_str.columns

Index(['building_id', 'district_id', 'vdcmun_id', 'ward_id',
       'count_floors_pre_eq', 'count_floors_post_eq', 'age_building',
       'plinth_area_sq_ft', 'height_ft_pre_eq', 'height_ft_post_eq',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'condition_post_eq'],
      dtype='object')

In [9]:
X = df[['area_assesed', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id']]
y = df.damage_grade

In [11]:
model = RandomForestClassifier(max_depth=16, n_estimators=100)
model.fit(X, y)
score = cross_val_score(model, X, y, scoring="neg_log_loss", cv=5, n_jobs=-1)
print(score)

ValueError: could not convert string to float: 'Both'

In [28]:
param = {'max_depth': np.arange(2,20), 'n_estimators': [50, 70, 100]}
clf = GridSearchCV(model, param, scoring='neg_log_loss', return_train_score=True)
clf.fit(X, y)
clf.best_params_

{'max_depth': 16, 'n_estimators': 100}